In [ ]:
import boto3
from boto3.s3.transfer import TransferConfig

# Upload the files in S3

In [ ]:
BUCKET = "<your bucket>"
REGION = "<your region>"
DATA_DIR = "<folder with your data>"
PROFILE = "<your profile>"

In [ ]:
def upload_to_s3(file_name, key, bucket=BUCKET, region=REGION):

    # Multipart upload config
    config = TransferConfig(
        multipart_threshold=1024 * 25,
        max_concurrency=10,
        multipart_chunksize=1024 * 25,
        use_threads=True,
    )
    
    # I have separate profiles in aws so I have to define one here
    # in order to authenticate myself to my account
    # If you do not have multiple profiles, just remove the argument
    session = boto3.Session(profile_name=PROFILE)
    
    s3 = session.client("s3", region_name=region)
    
    # Upload file to S3
    s3.upload_file(file_name, bucket, key, Config=config)

In [ ]:
from os import listdir
from os.path import isfile, join
data_files = [f for f in listdir(DATA_DIR) if isfile(join(DATA_DIR, f))]

In [ ]:
for file_name in data_files:
    file_path = join(DATA_DIR, file_name)
    upload_to_s3(file_path, file_name, bucket=BUCKET, region=REGION)

# Ask Amazon Rekognition

## Detect Labels

In [ ]:
# From Alex Casalboni: https://gist.github.com/alexcasalboni/0f21a1889f09760f8981b643326730ff
def detect_labels(bucket, key, max_labels=10, min_confidence=95, region=REGION):
    
    # I have separate profiles in aws so I have to define one here
    # in order to authenticate myself to my account
    # If you do not have multiple profiles, just remove the argument
    session = boto3.Session(profile_name=PROFILE)
    
    rekognition = session.client("rekognition", region)
    response = rekognition.detect_labels(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        MaxLabels=max_labels,
        MinConfidence=min_confidence,
    )
    return response.get('Labels', None)

In [ ]:
labels = detect_labels(BUCKET, "new_york.jpeg")

In [ ]:
labels

In [ ]:
for label in labels:
    print(f"{label['Name']} - {label['Confidence']}")

## Facial Analysis

In [ ]:
# From Alex Casalboni: https://gist.github.com/alexcasalboni/0f21a1889f09760f8981b643326730ff
def detect_faces(bucket, key, attributes=['ALL'], region=REGION):
    
    # I have separate profiles in aws so I have to define one here
    # in order to authenticate myself to my account
    # If you do not have multiple profiles, just remove the argument
    session = boto3.Session(profile_name='<profile bname>')
    
    rekognition = session.client("rekognition", region)
    response = rekognition.detect_faces(
        Image={
            "S3Object": {
                "Bucket": bucket,
                "Name": key,
            }
        },
        Attributes=attributes,
    )
    return response.get('FaceDetails', None)

In [ ]:
labels = detect_labels(BUCKET, "kostas.jpg")

In [ ]:
for label in labels:
    print(f"{label['Name']} - {label['Confidence']}")

In [ ]:
labels